In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_

import timm
import model.swin_v1 as swin

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
unique_idx = '4'

batch_size = 512
num_workers = 8
img_size = 224
patch_size = 16
dropout = 0.1

model_name = 'swin_tiny_patch4_window7_224.ms_in22k'
pretrained = False
num_classes = 10

device = f'cuda:{unique_idx}'

label_smoothing = 0.0
learning_rate = 1e-5
epochs = 30

model_path = f'../models/swin/finetune_model:{unique_idx}.pth'  # 모델 저장 경로

In [3]:
# 데이터 증강을 위한 전처리
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(img_size),  # 무작위 크기 및 비율로 자르기
    transforms.AutoAugment(AutoAugmentPolicy.CIFAR10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=transform_train)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)

testset = torchvision.datasets.CIFAR10(root='../data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
model1 = swin.SwinTransformer(img_size=224, num_classes=num_classes).to(device)
model2 = swin.SwinTransformer(img_size=224)
model2.load_state_dict(torch.load('../models/swin/model.pth', map_location=device))
model2.classifier = nn.Linear(model2.classifier.in_features, num_classes)
model2.to(device)
model3 = timm.create_model(model_name=model_name, 
                          pretrained=False, 
                          num_classes=num_classes).to(device)
model4 = timm.create_model(model_name=model_name, 
                          pretrained=True, 
                          num_classes=num_classes).to(device)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

optimizer1 = optim.AdamW(model1.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer2 = optim.AdamW(model2.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer3 = optim.AdamW(model3.parameters(), lr=learning_rate, weight_decay=1e-8)
optimizer4 = optim.AdamW(model4.parameters(), lr=learning_rate, weight_decay=1e-8)

In [6]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model1.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer1.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model1(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer1)
        clip_grad_norm_(model1.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer1)
        scaler.update()
        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model1.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model1(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

Epoch 1:  18%|█▊        | 18/98 [00:12<00:46,  1.72it/s]

In [ ]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model2.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer2.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model2(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer2)
        clip_grad_norm_(model2.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer2)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)
 
    model2.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model2(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)

In [ ]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model3.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer3.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model3(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer3)
        clip_grad_norm_(model3.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer3)
        scaler.update()
        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model3.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model3(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

In [ ]:
training_time = 0
losses = []
val_losses = []

# GradScaler 초기화
scaler = GradScaler()

for epoch in range(epochs):
    model4.train()
    start_time = time.time()
    running_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch + 1}")
    
    for i, data in pbar:
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer4.zero_grad()

        # AutoCast 적용
        with autocast():
            outputs = model4(inputs)
            loss = criterion(outputs, labels)

        # 스케일링된 그라디언트 계산
        scaler.scale(loss).backward()

        # 그라디언트 클리핑 전에 스케일링 제거
        scaler.unscale_(optimizer4)
        clip_grad_norm_(model4.parameters(), max_norm=1.0)

        # 옵티마이저 스텝 및 스케일러 업데이트
        scaler.step(optimizer4)
        scaler.update()

        # scheduler.step()
        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    losses.append(epoch_loss)

 
    model4.eval()
    val_loss = 0.0
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model4(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    val_loss /= len(test_loader)
    val_losses.append(val_loss)

    epoch_duration = time.time() - start_time
    training_time += epoch_duration
    
    text = f'\tLoss: {epoch_loss}, Val Loss: {val_loss}, Duration: {epoch_duration:.2f} sec'
    print(text)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 예측 수행 및 레이블 저장
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model4(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 혼동 행렬 생성
cm = confusion_matrix(all_labels, all_preds)

# 예측과 실제 레이블
y_true = all_labels  # 실제 레이블
y_pred = all_preds  # 모델에 의해 예측된 레이블

# 전체 데이터셋에 대한 정확도
accuracy = accuracy_score(y_true, y_pred)

# 평균 정밀도, 리콜, F1-Score ('weighted')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

# 판다스 데이터프레임으로 결과 정리
performance_metrics = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Value': [accuracy, precision, recall, f1_score]
})

# 데이터프레임 출력
display(performance_metrics)